# UQ (Uncertainty Quantification) Examples

In [ ]:
import os
os.chdir('/home/rarutter/development/dvm-dos-tem')

In [ ]:
import uncertainpy as un
import chaospy as cp                       # To create distributions
import numpy as np                         # For the time array
from scipy.integrate import odeint         # To integrate our equation
import matplotlib.pyplot as plt
import math
import json
import subprocess
import sys
import netCDF4 as nc
import scripts.output_utils as ou
%matplotlib inline

In [ ]:
PARAMS = {'cmax': [250,400], 'cfall(0)': [0.02,0.2]}
VAR = 'GPP'
PFTNUM = 1
CMTNUM = 4
PXx = 0; PXy = 0
H_RUNFOLDER = '.'
#H_RUNFOLDER = '../dvmdostem-workflows/uncertainty_quantification'

In [ ]:
# Cleanup
!rm -r {H_RUNFOLDER}

# Place to work
!setup_working_directory.py --input-data-path \
/data/input-catalog/cru-ts40_ar5_rcp85_ncar-ccsm4_CALM_Toolik_LTER_10x10/

# Run mask
!runmask-util.py --reset --yx {PXy} {PXx} {D_RUNFOLDER}/run-mask.nc

# Outputs
!outspec_utils.py {D_RUNFOLDER}/config/output_spec.csv --on GPP m p

!outspec_utils.py {D_RUNFOLDER}/config/output_spec.csv --on CMTNUM y

# Config, enable eq outputs
CONFIG_FILE = H_RUNFOLDER + '/config/config.js'

# Read the existing data into memory
with open(CONFIG_FILE, 'r') as f:
    config = json.load(f)

# Modify it
config['IO']['output_nc_eq'] = 1

# Write it back..
with open(CONFIG_FILE, 'w') as f:
    json.dump(config, f, indent=2)

In [ ]:
def adjust_param(param_name, param_value):
    data = !scripts/param_util.py --dump-block-to-json {H_RUNFOLDER}/default_parameters/cmt_calparbgc.txt {CMTNUM}
    
    jdata = json.loads(data[0])

    pft = 'pft{}'.format(PFTNUM)
    jdata[pft][param_name] = param_value

    with open("tmp_json.json", 'w') as f:
        json.dump(jdata, f)

    new_data = !scripts/param_util.py --fmt-block-from-json tmp_json.json {H_RUNFOLDER}/default_parameters/cmt_calparbgc.txt

    with open('{:}/parameters/cmt_calparbgc.txt'.format(H_RUNFOLDER), 'w') as f:
        # make sure to add newlines!
        f.write('\n'.join(new_data))

In [ ]:
def run_dvmdostem(**params_and_values):
    
    #Find and replace the parameters in the parameter file with the provided value
    for param in params_and_values:
        print("Modifying parameter {}".format(param))
        adjust_param(param, params_and_values[param])

    #Run dvmdostem
    run_result = !./dvmdostem -l fatal -p 10 -e 50 -s 20 -t 0 -n 0 --force-cmt {CMTNUM}

    #Load specified variable output and return. For now, the output needs to be (time,PFT,Y,X)
    var_output = collect_outputs()

    return None, var_output

In [ ]:
def collect_outputs():
    # Get the model output
    ds = nc.Dataset('{}/output/{}_monthly_eq.nc'.format(H_RUNFOLDER,VAR))
    print(ds)
    var_data = ds.variables[VAR][:]
    print(var_data.shape)

    #yr_gpp = ou.sum_monthly_flux_to_yearly(gpp)
    #output_data = yr_gpp[-1:,PFTNUM,PXy,PXx]
    
    #Veg var by PFT
    output_data = var_data[-60:,PFTNUM,PXy,PXx]

    #Soil var, single
    #output_data = var_data[-60:,PXy,PXx]
    
    print(output_data.shape)
    print(output_data)
    
    # Get the parameter value for the run
    #paramdata = !scripts/param_util.py --dump-block-to-json {H_RUNFOLDER}/parameters/cmt_calparbgc.txt {CMTNUM}
    #jparamdata = json.loads(paramdata[0])
    #pft = 'pft{}'.format(PFTNUM)
    #run_param_value = jparamdata[pft][PARAM]
    
    return output_data

In [ ]:
# Create a model from the calling function and add labels
model = un.Model(run=run_dvmdostem, labels=["Months", VAR])
model.labels

In [ ]:
# Create the parameter dictionary with distributions
# Arbitray numbers for testing
parameters = {}
for key in PARAMS:
    parameters[key] = cp.Uniform(PARAMS[key][0], PARAMS[key][1])
print(parameters)


In [ ]:
# Set up the uncertainty quantification
# Temporarily forced to sequential because separate working directories are not implemented
UQ = un.UncertaintyQuantification(model=model, parameters=parameters, CPUs=None)

# Perform the uncertainty quantification using
# polynomial chaos with point collocation (by default)
# We set the seed to easier be able to reproduce the result
data = UQ.quantify(seed=10)

In [ ]:
key="run_dvmdostem"
run_dvmdostem = un.Data("data/run_dvmdostem.h5")
time = run_dvmdostem[key].time
print(time.shape)
mean = run_dvmdostem[key].mean
print(mean.shape)
print(mean)
variance = run_dvmdostem[key].variance
print(variance)
percentile_5 = run_dvmdostem[key].percentile_5
percentile_95 = run_dvmdostem[key].percentile_95
sensitivity = run_dvmdostem[key].sobol_first

labels = run_dvmdostem.get_labels(key)
xlabel, ylabel = labels

In [ ]:
!ncdump data/run_dvmdostem.h5

In [ ]:
time = range(0,60)
plt.plot(time,mean,lw=2,color='black')
plt.fill_between(time, percentile_5, percentile_95, color='b', alpha=0.2)
plt.plot(time,variance,lw=1,color='orange')
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.rcParams['figure.figsize'][0] = 40
plt.legend(['mean','variance'])